In [19]:
import io
import json
import re
import hashlib
from collections import defaultdict
from enum import Enum, auto
from pathlib import Path

import fitz
import numpy as np
from IPython.display import display
from PIL import Image

In [20]:
def extract_pdf_pages_data(pdf_path):
  """
  Extract text and images from all pages of a PDF.

  Returns:
      [
        {
          "text": str,
          "images": [PIL.Image.Image, ...]
        },
        ...
      ]
  """
  pdf_path = Path("./data/poster-pdfs") / pdf_path
  doc = fitz.open(pdf_path)

  pages_data = []

  for page_index in range(doc.page_count):
    page = doc.load_page(page_index)

    # ---- TEXT ----
    blocks = page.get_text("blocks")
    _text_for_sorting = []

    for b in blocks:
      x0, y0, x1, y1, text, *_ = b
      if text.strip():
        _text_for_sorting.append((y0, x0, text))

    text = "".join(
        t[2] for t in sorted(_text_for_sorting, key=lambda t: t[:-1])
    )

    # ---- IMAGES ----
    _images_for_sorting = []

    for img in page.get_images(full=True):
      xref = img[0]
      img_name = img[7]

      bbox = page.get_image_bbox(img_name)
      y0, x0 = bbox.y0, bbox.x0

      pix = fitz.Pixmap(doc, xref)

      # Ensure RGB (PNG-safe)
      if pix.colorspace is None or pix.colorspace.n != 3:
        pix = fitz.Pixmap(fitz.csRGB, pix)

      pil_img = Image.open(io.BytesIO(pix.tobytes("png")))

      _images_for_sorting.append((y0, x0, pil_img))

      pix = None

    images = [
        t[2] for t in sorted(_images_for_sorting, key=lambda t: t[:-1])
    ]

    pages_data.append({
        "text": text,
        "images": images
    })

  doc.close()
  return pages_data

In [21]:
class Poster:
  def __init__(self, pdf_path: str, page_no: int, code: str, image):
    self.pdf_path = pdf_path
    self.page_no = page_no
    self.code = code.strip()
    self.image = image
    self.size = image.size  # (w, h)

    pdf_name = Path(pdf_path).stem
    safe_code = re.sub(r"[^\w\-]", "_", self.code)

    h = hashlib.md5(
        image.tobytes()
    ).hexdigest()[:5]

    self.image_file = f"{pdf_name}--p{page_no}--{safe_code}--{h}.jpg"

  def dictify(self):
    return {
        "page_no": self.page_no,
        "code": self.code,
        "image_file": self.image_file,
        "size": self.size,
    }

  def __repr__(self):
    return f"<Poster pdf={self.pdf_path}, page={self.page_no}, code={self.code}>"

In [22]:
class PDF:
  def __init__(self, pdf_path: str, pdf_info: dict, posters: list):
    self.path = pdf_path
    self.id = Path(pdf_path).stem
    self.category = pdf_info["category"]
    self.posters = posters  # list[Poster]

  def dictify(self, include_posters: bool):
    obj = {
        "id": self.id,
        "path": self.path,
        "category": self.category,
        "total_posters": len(self.posters),
        "thumbnail": self.posters[0].image_file,
    }

    if include_posters:
      obj["posters"] = [p.dictify() for p in self.posters]

    return obj

In [23]:
class TextExtractMode(Enum):
  LAST_LINE = auto()
  LAST_LINE_AFTER_COLON = auto()
  SECOND_LAST_LINE = auto()

# pdf_registry

In [24]:
pdfs_multi_per_page = {
    "multi-per-page/01_Radha_Krishna.pdf": {
        "category": "Radha Krishna",
    },
    "multi-per-page/02_Radha_Krishna_Vertical.pdf": {
        "category": "Radha Krishna",
    },
    "multi-per-page/03_Mixed.pdf": {
        "category": "Mixed",
    },
    "multi-per-page/04_Ganesh.pdf": {
        "category": "Ganesh",
    },
    "multi-per-page/08_Ambedkar_Buddha.pdf": {
        "category": "Ambedkar Buddha",
    },
    "multi-per-page/08_Horse.pdf": {
        "category": "Horse",
    },
    "multi-per-page/09_Ambedkar.pdf": {
        "category": "Ambedkar",
    },
    "multi-per-page/38_Nature.pdf": {
        "category": "Nature",
    },
}

pdfs_single_per_page = {
    "1-per-page/02_Ram.pdf": {
        "text_mode": TextExtractMode.SECOND_LAST_LINE,
        "category": "Ram",
    },
    "1-per-page/29_Chinese_Scenery_Modern.pdf": {
        "text_mode": TextExtractMode.LAST_LINE_AFTER_COLON,
        "category": "Chinese Scenery",
    },
    "1-per-page/30_Chinese_Scenery_Modern.pdf": {
        "text_mode": TextExtractMode.LAST_LINE_AFTER_COLON,
        "category": "Chinese Scenery",
    },
    "1-per-page/31_Chinese_Scenery_Modern.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Chinese Scenery",
    },
    "1-per-page/40_Mahakal.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Mahakal",
    },
    "1-per-page/45_Tirupati_Balaji.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Tirupati Balaji",
    },
    "1-per-page/46_Ram_Darbar.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Ram Darbar",
    },
    "1-per-page/47_Shiva_Family.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Shiva Family",
    },
    "1-per-page/48_Hanuman.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Hanuman",
    },
    "1-per-page/51_Buddha.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Buddha",
    },
    "1-per-page/61_Mixed_Gods.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Mixed Gods",
    },
    "1-per-page/72_Buddha.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Buddha",
    },
    "1-per-page/75_Khatu_Shyam.pdf": {
        "text_mode": TextExtractMode.LAST_LINE,
        "category": "Khatu Shyam",
    },
}

pdfs_incremental = {
    "ocr-needed/39_Modern.pdf": {
        "start_code": 25284,
        "prefix": "",
        "category": "Modern",
    },
    "ocr-needed/43_Modern.pdf": {
        "start_code": 25725,
        "prefix": "",
        "category": "Modern",
    },
    "ocr-needed/44_Modern.pdf": {
        "start_code": 25569,
        "prefix": "",
        "category": "Modern",
    },
    "ocr-needed/49_LGS.pdf": {
        "start_code": 1001,
        "prefix": "LGS",
        "category": "Lakshmi Ganesh Saraswati",
    },
}

In [25]:
PDF_REGISTRY = {
    **pdfs_multi_per_page,
    **pdfs_single_per_page,
    **pdfs_incremental,
}

In [36]:
CATEGORY_REGISTRY = [
    {
        "name": "Radha Krishna",
        "description": (
            "घर में प्रेम, सामंजस्य और भक्ति का संचार करता है। "
            "राधा कृष्ण पोस्टर दिव्य प्रेम और भावनात्मक संतुलन का प्रतीक हैं।"
        ),
    },
    {
        "name": "Chinese Scenery",
        "description": (
            "आपके स्थान में शांति और प्राकृतिक संतुलन लाता है। "
            "चीनी प्राकृतिक दृश्य कला शांति, समृद्धि और सजगता को बढ़ावा देती है।"
        ),
    },
    {
        "name": "Ganesh",
        "description": (
            "भगवान गणेश बाधाओं को दूर करते हैं और सफलता आकर्षित करते हैं। "
            "नए आरंभ और सकारात्मक ऊर्जा के लिए आदर्श पोस्टर।"
        ),
    },
    {
        "name": "Ambedkar Buddha",
        "description": (
            "ज्ञान, समानता और आंतरिक शक्ति की प्रेरक छवि। "
            "प्रेरणा और विचारशील जीवन के लिए उपयुक्त।"
        ),
    },
    {
        "name": "Horse",
        "description": (
            "गति, शक्ति और प्रगति का प्रतीक। "
            "घोड़े के पोस्टर ऊर्जा बढ़ाते हैं और आगे बढ़ने की प्रेरणा देते हैं।"
        ),
    },
    {
        "name": "Ambedkar",
        "description": (
            "बुद्धिमत्ता, साहस और सामाजिक न्याय का सम्मान। "
            "प्रेरणा और आत्मसम्मान के लिए अर्थपूर्ण पोस्टर।"
        ),
    },
    {
        "name": "Nature",
        "description": (
            "शांति और ताजगी से जुड़ने का माध्यम। "
            "प्राकृतिक पोस्टर तनाव कम करते हैं और दृश्य संतुलन लाते हैं।"
        ),
    },
    {
        "name": "Ram",
        "description": (
            "भगवान राम धर्म, अनुशासन और नैतिक शक्ति के प्रतीक हैं। "
            "शांति और पारिवारिक मूल्यों के लिए आदर्श।"
        ),
    },
    {
        "name": "Mahakal",
        "description": (
            "महाकाल समय, शक्ति और निर्भयता का प्रतीक हैं। "
            "आत्मविश्वास बढ़ाने वाली सशक्त आध्यात्मिक उपस्थिति।"
        ),
    },
    {
        "name": "Tirupati Balaji",
        "description": (
            "धन, भक्ति और मनोकामना पूर्ति से जुड़े। "
            "घर और कार्यस्थल के लिए अत्यंत शुभ।"
        ),
    },
    {
        "name": "Ram Darbar",
        "description": (
            "आदर्श पारिवारिक मूल्यों और नेतृत्व का संपूर्ण चित्र। "
            "एकता, भक्ति और सामंजस्य को प्रोत्साहित करता है।"
        ),
    },
    {
        "name": "Shiva Family",
        "description": (
            "शक्ति, करुणा और परिवार के संतुलन का प्रतीक। "
            "घर में आध्यात्मिक स्थिरता लाता है।"
        ),
    },
    {
        "name": "Hanuman",
        "description": (
            "भक्ति, शक्ति और सुरक्षा का प्रतीक। "
            "साहस, एकाग्रता और भय से मुक्ति के लिए उपयुक्त।"
        ),
    },
    {
        "name": "Buddha",
        "description": (
            "शांति, सजगता और ज्ञानोदय का प्रतीक। "
            "ध्यान कक्ष और शांत वातावरण के लिए उपयुक्त।"
        ),
    },
    {
        "name": "Mixed Gods",
        "description": (
            "विभिन्न देवी-देवताओं की सामूहिक दिव्य ऊर्जा। "
            "एक ही चित्र में संपूर्ण आध्यात्मिक संतुलन के लिए।"
        ),
    },
    {
        "name": "Khatu Shyam",
        "description": (
            "आस्था, चमत्कार और अटूट भक्ति के लिए प्रसिद्ध। "
            "भक्तों के लिए आशा और भावनात्मक शक्ति प्रदान करता है।"
        ),
    },
    {
        "name": "Modern",
        "description": (
            "आधुनिक कला जो शैली और व्यक्तित्व जोड़ती है। "
            "आधुनिक इंटीरियर और रचनात्मक स्थानों के लिए आदर्श।"
        ),
    },
    {
        "name": "Lakshmi Ganesh Saraswati",
        "description": (
            "धन, विद्या और सफलता का शक्तिशाली संगम। "
            "समृद्धि और शिक्षा के लिए अत्यंत लाभकारी।"
        ),
    },
    {
        "name": "Mixed",
        "description": (
            "जिसका कोई निश्चित विषय नहीं होता। "
            "इसमें गणेश, बुद्ध, घोड़े, प्रकृति या आधुनिक कला कुछ भी हो सकता है।"
        ),
    },
]

# multi_per_page

In [27]:
def extract_multi_per_page(pdf_path, options):
  data = extract_pdf_pages_data(pdf_path)
  result = []

  for page_idx, page in enumerate(data[1:], start=2):
    images = page["images"]
    n = len(images)
    codes = page["text"].splitlines()[:-1][-n:]

    for code, image in zip(codes, images):
      result.append(
          Poster(pdf_path, page_idx, code, image)
      )

  return result

# single_per_page

In [28]:
def extract_single_per_page(pdf_path, options):
  mode = options["text_mode"]

  data = extract_pdf_pages_data(pdf_path)
  result = []

  for page_idx, page in enumerate(data, start=1):
    lines = page["text"].splitlines()
    if not lines:
      continue

    if mode == TextExtractMode.LAST_LINE:
      code = lines[-1]
    elif mode == TextExtractMode.LAST_LINE_AFTER_COLON:
      code = lines[-1].split(":", 1)[1].strip()
    elif mode == TextExtractMode.SECOND_LAST_LINE:
      code = lines[-2]

    image = page["images"][-1]

    result.append(
        Poster(pdf_path, page_idx, code, image)
    )

  return result

# incremental_ocr_needed

In [29]:
def extract_incremental(pdf_path, options):
  start_code = options["start_code"]
  prefix = options.get("prefix", "")

  data = extract_pdf_pages_data(pdf_path)
  result = []

  code = start_code

  for page_idx, page in enumerate(data, start=1):
    image = page["images"][-1]
    full_code = f"{prefix}-{code}" if prefix else str(code)

    result.append(
        Poster(pdf_path, page_idx, full_code, image)
    )

    code += 1

  return result

# generate_all_posters 

In [30]:
def generate_all_posters():
  pipelines = [
      (pdfs_multi_per_page, extract_multi_per_page),
      (pdfs_single_per_page, extract_single_per_page),
      (pdfs_incremental, extract_incremental),
  ]

  pdfs = []
  total_posters = 0

  for registry, extractor in pipelines:
    for pdf_path, options in registry.items():
      print(f"Processing: {pdf_path}")
      posters = extractor(pdf_path, options)

      pdf = PDF(
          pdf_path=pdf_path,
          pdf_info=options,
          posters=posters,
      )
      pdfs.append(pdf)

      total_posters += len(posters)

  print(f"Total posters: {total_posters}")
  return pdfs

In [31]:
pdfs = generate_all_posters()

Processing: multi-per-page/01_Radha_Krishna.pdf
Processing: multi-per-page/02_Radha_Krishna_Vertical.pdf
Processing: multi-per-page/03_Mixed.pdf
Processing: multi-per-page/04_Ganesh.pdf
Processing: multi-per-page/08_Ambedkar_Buddha.pdf
Processing: multi-per-page/08_Horse.pdf
Processing: multi-per-page/09_Ambedkar.pdf
Processing: multi-per-page/38_Nature.pdf
Processing: 1-per-page/02_Ram.pdf
Processing: 1-per-page/29_Chinese_Scenery_Modern.pdf
Processing: 1-per-page/30_Chinese_Scenery_Modern.pdf
Processing: 1-per-page/31_Chinese_Scenery_Modern.pdf
Processing: 1-per-page/40_Mahakal.pdf
Processing: 1-per-page/45_Tirupati_Balaji.pdf
Processing: 1-per-page/46_Ram_Darbar.pdf
Processing: 1-per-page/47_Shiva_Family.pdf
Processing: 1-per-page/48_Hanuman.pdf
Processing: 1-per-page/51_Buddha.pdf
Processing: 1-per-page/61_Mixed_Gods.pdf
Processing: 1-per-page/72_Buddha.pdf
Processing: 1-per-page/75_Khatu_Shyam.pdf
Processing: ocr-needed/39_Modern.pdf
Processing: ocr-needed/43_Modern.pdf
Processing

# save_poster_images

In [32]:
def save_poster_images(
    pdfs,
    output_dir="frontend/public/poster-images",
):
  output_dir = Path(output_dir)
  output_dir.mkdir(parents=True, exist_ok=True)

  for pdf in pdfs:
    for poster in pdf.posters:
      path = output_dir / poster.image_file

      if path.exists():
        continue

      poster.image.convert("RGB").save(
          path,
          format="JPEG",
          quality=85,
          subsampling=2,
          optimize=True,
          progressive=True,
      )

In [33]:
save_poster_images(pdfs)

# generate_pdf_json

In [34]:
def generate_pdf_json(
    pdfs,
    output_dir="frontend/public/pdfs-data",
):
  output_dir = Path(output_dir)
  output_dir.mkdir(parents=True, exist_ok=True)

  pdfs_metadata = []

  for pdf in pdfs:
    # full PDF data
    with open(output_dir / f"{pdf.id}.json", "w", encoding="utf-8") as f:
      json.dump(pdf.dictify(include_posters=True), f, indent=2, ensure_ascii=False)

    # metadata entry
    pdfs_metadata.append(pdf.dictify(include_posters=False))

  metadata = {
      "pdfs": pdfs_metadata,
      "categories": CATEGORY_REGISTRY,
  }

  with open(output_dir / "metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

In [37]:
generate_pdf_json(pdfs)